In [2]:
import numpy as np
import matplotlib.pyplot as plt

import xtrack as xt
import xcoll as xc
import xobjects as xo
import xpart as xp
from pathlib import Path
import pickle

In [3]:
def generate_multivariate_uniform_ring(
        ndims: int, 
        nsamples: int=1, 
        inner_radius: float=0.0, 
        outer_radius:float=1.0
        ):
    
    nsamples = int(nsamples)

    _ = np.random.normal(size=(nsamples, ndims))
    _ = _ / np.linalg.norm(_, axis=-1, keepdims=True)
    __ = np.random.uniform(size=(nsamples, ))
    __ = (__ * (outer_radius**ndims - inner_radius**ndims) + inner_radius**ndims)**(1/ndims)

    return _ * __[:, None]

In [4]:
def generate_longitudinal_slice(line, num_particles, cut, sigma_z, upper_cut=None):
    zeta = []
    delta = []
    num = 0
    step = int(1.e7)
    while True:
        this_zeta, this_delta = xp.generate_longitudinal_coordinates(num_particles=step,
                                    distribution='gaussian', sigma_z=sigma_z, line=line)
        amp = np.sqrt((this_zeta/this_zeta.std())**2 + (this_delta/this_delta.std())**2)
        mask = amp >= cut
        if upper_cut is not None:
            mask = mask & (amp <= upper_cut)
        zeta = [*zeta, *this_zeta[mask]]
        delta = [*delta, *this_delta[mask]]
        num += len(np.where(mask)[0])
        if num >= num_particles:
            zeta = zeta[:num_particles]
            delta = delta[:num_particles]
            break
    return zeta, delta

In [5]:
path_injection_lines = Path('/eos/user/l/lpauwels/ht_condor_sps_tracking_results/injection_lines')
line = xt.Line.from_json(Path(path_injection_lines, 'sps_lhc_q20_rf_with_ap.json')) #Line with magnet errors
#line = xt.Line.from_json(Path(path_injection_lines, 'injection_thin_approx_ap_with_deferred_expressions.json')) #Line without magnet errors

Loading line from dict:   0%|          | 0/28458 [00:00<?, ?it/s]

Done loading line from dict.           


In [18]:
shell = 'x'
path_out = Path('/eos/user/l/lpauwels/ht_condor_sps_tracking_results/ripple_tests/ripple_tests_results')
outfile_name = 'errors_shell_x_12_50'

In [7]:
num_turns = 500_000
num_particles = 50

nemitt_x = 3.5e-6
nemitt_y = 3.5e-6

In [8]:
print('Install collimator')
coll = xc.EverestCollimator(length=1.83, gap=5, material=xc.materials.Carbon) # length is 1.83
line.collimators.install('tcsm.51932', coll)

#Make aperture for collimator and update line
coll_ap = xt.LimitRectEllipse(a=0.05, b=0.05, max_x=0.05, max_y=0.05) 
coll_ap_names = ['tcsm.51932_aper_upstream', 'tcsm.51932_aper_downstream']
coll_ap_idx = [line.element_names.index('tcsm.51932'), line.element_names.index('tcsm.51932') + 1]

max_length = max(max(map(len, line.element_names)), max(map(len, coll_ap_names)))
element_names = np.array(line.element_names, dtype=f'<U{max_length}')
names = np.array(coll_ap_names, dtype=f'<U{max_length}')
element_names = np.insert(element_names, coll_ap_idx, coll_ap_names)

insert_colls = {name: coll_ap for name in coll_ap_names}

line.element_names = element_names.tolist()
line.element_dict = {**line.element_dict, **insert_colls}

Install collimator


Slicing line:   0%|          | 0/28458 [00:00<?, ?it/s]

In [8]:
df_with_coll = line.check_aperture()

Checking aperture:   0%|          | 0/28455 [00:00<?, ?it/s]

Done checking aperture.           
0 thin elements miss associated aperture (upstream):
[]
0 thick elements miss associated aperture (upstream or downstream):
[]


In [ ]:
line.build_tracker()
tw = line.twiss()
sigma_x = np.sqrt(nemitt_x*tw.betx/line.particle_ref.gamma0)
sigma_y = np.sqrt(nemitt_y*tw.bety/line.particle_ref.gamma0)

line.collimators.assign_optics(twiss=tw, nemitt_x=nemitt_x, nemitt_y=nemitt_y)
line.optimize_for_tracking()

# Start interaction record
#impacts = xc.InteractionRecord.start(line=line, record_impacts=True)

Disable xdeps expressions
Replance slices with equivalent elements
Remove markers
Remove inactive multipoles
Merge consecutive multipoles
Remove redundant apertures
Remove zero length drifts
Merge consecutive drifts
Use simple bends
Use simple quadrupoles
Rebuild tracker data


ValueError: No Xcoll elements in line!

# Ripple preparation

In [8]:
#Ripple information
kqf_amplitudes = np.array([9.7892e-7])
kqd_amplitudes = np.array([9.6865e-7])
kqf_phases=np.array([0.5564486])
kqd_phases=np.array([0.47329223])
ripple_freqs=np.array([50.])

In [9]:
def get_k_ripple_summed_signal(num_turns, ripple_periods, kqf_amplitudes, kqd_amplitudes,
                                   kqf_phases, kqd_phases):
    """
    Generate noise signal on top of kqf/kqd values, with desired ripple periods and amplitudes.
    Phase and frequencies unit must correspond to where it is used, e.g turns
    
    Parameters:
    -----------
    ripple_periods : np.ndarray
        floats containing the ripple periods of the noise frequencies
    kqf_amplitudes : np.ndarray
        ripple amplitudes for desired frequencies of kqf --> obtained from normalized FFT spectrum of IQD and IQF. 
        Default without 50 Hz compensation is 1e-6
    kqd_amplitudes : list
        ripple amplitudes for desired frequencies of kqd --> obtained from normalized FFT spectrum of IQD and IQF. 
        Default without 50 Hz compensation is 1e-6
    kqf_phases : np.ndarray
        ripple phase for desired frequencies of kqf --> obtained from normalized FFT spectrum of IQD and IQF. 
    kqd_phases : list
        ripple phases for desired frequencies of kqd --> obtained from normalized FFT spectrum of IQD and IQF. 

    Returns:
    --------
    k_ripple_values : np.ndarray
        focusing quadrupole values corresponding to modulate Qx according to dq (if chosen plane)
    """

    turns = np.arange(1, num_turns+1)
    kqf_signals = np.zeros([len(ripple_periods), len(turns)])
    kqd_signals = np.zeros([len(ripple_periods), len(turns)])
    for i, ripple_period in enumerate(ripple_periods):
        kqf_signals[i, :] = kqf_amplitudes[i] * np.sin(2 * np.pi * turns / ripple_period + kqf_phases[i])
        kqd_signals[i, :] = kqd_amplitudes[i] * np.sin(2 * np.pi * turns / ripple_period + kqd_phases[i])

    # Sum the signal
    kqf_ripple = np.sum(kqf_signals, axis=0)
    kqd_ripple = np.sum(kqd_signals, axis=0)

    print('Generated kqf ripple of amplitudes {} and phases {} with ripple periods {}'.format(kqf_amplitudes, kqf_phases, ripple_periods))
    print('Generated kqd ripple of amplitudes {} and phases {} with ripple periods {}'.format(kqd_amplitudes, kqd_phases, ripple_periods))

    return kqf_ripple, kqd_ripple

In [10]:
# Create ripple in quadrupolar knobs, convert phases to turns
turns_per_sec = 1/tw.T_rev0
ripple_periods = (turns_per_sec/ripple_freqs).astype(int)  # number of turns particle makes during one ripple oscillation
kqf_phases_turns = kqf_phases * turns_per_sec # convert time domain to turn domain, i.e. multiply with turns/sec
kqd_phases_turns = kqd_phases * turns_per_sec # convert time domain to turn domain, i.e. multiply with turns/sec

#ripple_maker = Tune_Ripple_SPS(num_turns=num_turns) # qx0=self.qx0, qy0=self.qy0)
kqf_ripple, kqd_ripple = get_k_ripple_summed_signal(num_turns, ripple_periods, kqf_amplitudes, kqd_amplitudes, kqf_phases_turns, kqd_phases_turns)

Generated kqf ripple of amplitudes [9.7892e-07] and phases [24120.73842865] with ripple periods [866]
Generated kqd ripple of amplitudes [9.6865e-07] and phases [20516.10531529] with ripple periods [866]


In [11]:
# Save initial values
kqf0 = line.vars['kqf']._value
kqd0 = line.vars['kqd']._value

print('Quadrupolar knobs will oscillate with')
print('kqf =  {:.4e} +/- {:.3e}'.format(kqf0, max(kqf_ripple)))
print('kqd = {:.4e} +/- {:.3e}'.format(kqd0, max(kqd_ripple)))

Quadrupolar knobs will oscillate with
kqf =  1.1580e-02 +/- 9.789e-07
kqd = -1.1581e-02 +/- 9.686e-07


# Particles and tracking with ripple

In [32]:
line.scattering.disable()

if shell == 'l':
    x_norm = np.random.normal(size=num_particles)
    px_norm = np.random.normal(size=num_particles)
    y_norm = np.random.normal(size=num_particles)
    py_norm = np.random.normal(size=num_particles)

    zeta, delta = generate_longitudinal_slice(line, num_particles, cut=2.7, sigma_z=0.224)
    part = line.build_particles(x_norm=x_norm, px_norm=px_norm, y_norm=y_norm, py_norm=py_norm, nemitt_x=nemitt_x, nemitt_y=nemitt_y, zeta = zeta, delta = delta)
elif shell == 'x':
    x_norm = []
    px_norm = []
    while True:
        this_x_norm = np.random.normal(size=10_000_000)
        this_px_norm = np.random.normal(size=10_000_000)
        mask = np.sqrt(this_x_norm**2 + this_px_norm**2) >= 4
        x_norm.extend(this_x_norm[mask])
        px_norm.extend(this_px_norm[mask])
        if len(x_norm) >= num_particles:
            x_norm = np.array(x_norm[:num_particles])
            px_norm = np.array(px_norm[:num_particles])
            break

    y_norm = np.random.normal(size=num_particles)
    py_norm = np.random.normal(size=num_particles)
    zeta, delta = generate_longitudinal_slice(line, num_particles, cut=0, sigma_z=0.224)
    part = line.build_particles(x_norm=x_norm, px_norm=px_norm, y_norm=y_norm, py_norm=py_norm, nemitt_x=nemitt_x, nemitt_y=nemitt_y)

*** Maximum RMS bunch length 0.23610204667323867m.
... distance to target bunch length: -2.2400e-01
... distance to target bunch length: 5.5793e-03
... distance to target bunch length: 5.2397e-03
... distance to target bunch length: -6.3409e-03
... distance to target bunch length: 1.9992e-03
... distance to target bunch length: -3.8457e-04
... distance to target bunch length: 5.4240e-05
... distance to target bunch length: 1.2753e-06
... distance to target bunch length: -1.1273e-10
... distance to target bunch length: 1.2806e-07
--> Bunch length: 0.22399999988726663
--> Emittance: 0.16126789877093833


In [14]:
#Multicore
line.discard_tracker()
line.build_tracker(_context=xo.ContextCpu(omp_num_threads='auto'))

time = 0

# Track!
line.scattering.enable()
for turn in range(1, num_turns):
    if turn%5000 == 0:
        print(f'Turn {turn}')
        print(np.unique(part.state, return_counts=True))
    
    line.vars['kqf'] = kqf0 + kqf_ripple[turn-1]
    line.vars['kqd'] = kqd0 + kqd_ripple[turn-1]
    
    line.track(part, num_turns=1, time=True)
    time += line.time_last_track
    current_part_state = np.unique(part.state, return_counts=True)
    if 1 not in current_part_state[0]:
        break

print(f"Done tracking in {time:.1f}s.")
line.scattering.disable()

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Turn 5000
(array([1]), array([50]))
Turn 10000
(array([1]), array([50]))
Turn 15000
(array([1]), array([50]))
Turn 20000
(array([1]), array([50]))
Turn 25000
(array([1]), array([50]))
Turn 30000
(array([1]), array([50]))
Turn 35000
(array([1]), array([50]))
Turn 40000
(array([1]), array([50]))
Turn 45000
(array([1]), array([50]))
Turn 50000
(array([1]), array([50]))
Turn 55000
(array([1]), array([50]))
Turn 60000
(array([1]), array([50]))
Turn 65000
(array([1]), array([50]))
Turn 70000
(array([1]), array([50]))
Turn 75000
(array([1]), array([50]))
Turn 80000
(array([1]), array([50]))
Turn 85000
(array([1]), array([50]))
Turn 90000
(array([1]), array([50]))
Turn 95000
(array([1]), array([50]))
Turn 100000
(array([1]), array([50]))
Turn 105000
(array([1]), array([50]))
Turn 110000
(array([1]), array([50]))
Turn 115000
(array([1]), array([50]))
Turn 120000
(array([1]), array([50]))
Turn 125000
(array([1]), array([50]))
Tur

# Tracking without ripple

In [14]:
#For very far x shell distribution, you can generate uniform distributions in the x-px plane
p = generate_multivariate_uniform_ring(2, num_particles, 12, 13)
x_norm = p[:, 0]
px_norm = p[:, 1]
y_norm = np.random.normal(size=num_particles)
py_norm = np.random.normal(size=num_particles)

zeta, delta = generate_longitudinal_slice(line, num_particles, cut=0, sigma_z=0.224)
part = line.build_particles(
    x_norm=p[:, 0], px_norm=p[:, 1],
    y_norm=y_norm, py_norm=py_norm,
    nemitt_x=nemitt_x, nemitt_y=nemitt_y,
    zeta=zeta, delta=delta
)

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
*** Maximum RMS bunch length 0.23610204667323867m.
... distance to target bunch length: -2.2400e-01
... distance to target bunch length: 5.5793e-03
... distance to target bunch length: 5.2397e-03
... distance to target bunch length: -6.3409e-03
... distance to target bunch length: 1.9992e-03
... distance to target bunch length: -3.8457e-04
... distance to target bunch length: 5.4240e-05
... distance to target bunch length: 1.2753e-06
... distance to target bunch length: -1.1273e-10
... distance to target bunch length: 1.2806e-07
--> Bunch length: 0.2239999998872666
--> Emittance: 0.16126789877093822


In [15]:
line.discard_tracker()
line.build_tracker(_context=xo.ContextCpu(omp_num_threads='auto'))
line.scattering.enable()
line.track(part, num_turns=num_turns, time=True, with_progress=1)
line.scattering.disable()

Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
No xcoll elements found in line.


Tracking:   0%|          | 0/500000 [00:00<?, ?it/s]

No xcoll elements found in line.


In [16]:
np.unique(part.state, return_counts=True)

(array([0, 1]), array([ 2, 48]))

# Lossmaps

In [21]:
line.discard_tracker()
line.build_tracker(_context=xo.ContextCpu())
ThisLM = xc.LossMap(line, line_is_reversed=False, part=part)
ThisLM.to_json(file=Path(path_out, f'LM_{outfile_name}.json'))

with open(Path(path_out, f'particles_{outfile_name}.pkl'), 'wb') as f:
    pickle.dump(part.to_dict(), f)

Performing the aperture losses refinement.
